In [1]:
import time
import datetime
from model import *
from data import *
from tensorflow.python.client import device_lib


Using TensorFlow backend.


In [2]:
def myTrainGenerator(batch_size,train_path,image_folder,mask_folder,aug_dict,image_color_mode = "rgb",
                    mask_color_mode = "rgb",image_save_prefix  = "image",mask_save_prefix  = "mask",
                    flag_multi_class = False,num_class = 2,save_to_dir = "temp\\",target_size = (256,256),seed = 1):
    '''
    can generate image and mask at the same time
    use the same seed for image_datagen and mask_datagen to ensure the transformation for image and mask is the same
    if you want to visualize the results of generator, set save_to_dir = "your path"
    '''
    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)
    image_generator = image_datagen.flow_from_directory(
        train_path,
        classes = [image_folder],
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        seed = seed)
    mask_generator = mask_datagen.flow_from_directory(
        train_path,
        classes = [mask_folder],
        class_mode = None,
        color_mode = mask_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = mask_save_prefix,
        seed = seed)
    train_generator = zip(image_generator, mask_generator)
    for (img,mask) in train_generator:
        img,mask = myAdjustData(img,mask,flag_multi_class,num_class)
        yield (img,mask)

In [7]:
def myAdjustData(img,mask,flag_multi_class,num_class):
    if(flag_multi_class):
        img = img / 255
        mask = mask[:,:,:,0] if(len(mask.shape) == 4) else mask[:,:,0]
        new_mask = np.zeros(mask.shape + (num_class,))
        for i in range(num_class):
            #for one pixel in the image, find the class in mask and convert it into one-hot vector
            #index = np.where(mask == i)
            #index_mask = (index[0],index[1],index[2],np.zeros(len(index[0]),dtype = np.int64) + i) if (len(mask.shape) == 4) else (index[0],index[1],np.zeros(len(index[0]),dtype = np.int64) + i)
            #new_mask[index_mask] = 1
            new_mask[mask == i,i] = 1
        new_mask = np.reshape(new_mask,(new_mask.shape[0],new_mask.shape[1]*new_mask.shape[2],new_mask.shape[3])) if flag_multi_class else np.reshape(new_mask,(new_mask.shape[0]*new_mask.shape[1],new_mask.shape[2]))
        mask = new_mask
    elif(np.max(img) > 1):
        img[0][0][0][1] = 1
        img[0][0][0][2] = 0
        mask[0][0][0][1] = 1
        mask[0][0][0][1] = 0
        img = img / 255
        mask = mask /255
        mask[mask > 0.5] = 1
        mask[mask <= 0.5] = 0
    return (img,mask)

In [8]:


# Check the currently available GPUs
print(device_lib.list_local_devices())


'''
Get time
'''
now = time.time()
time_stamp = datetime.datetime.fromtimestamp(now).strftime('_%m_%d_%H_%M')


'''
This script train the model using the PNG images and labels, and save the model as hdf5 file
'''

image_folder = 'mri_images_30'
label_folder = 'mri_labels_30'
save_folder = 'model_checkpoint'
model_name = 'colon'

model_name = model_name + time_stamp

'''
Set the parameters and the model type for the training
'''
data_gen_args = dict(rotation_range=0.2,
					width_shift_range=0.05,
					height_shift_range=0.05,
					shear_range=0.05,
					zoom_range=0.05,
					horizontal_flip=True,
					fill_mode='nearest')

save_path = save_folder + '/' + model_name + '.hdf5'

myGene = myTrainGenerator(2,'Datasets',image_folder,label_folder,data_gen_args,save_to_dir = None)

model = unet()
# model = unet_lrelu()

model_checkpoint = ModelCheckpoint(save_path, monitor='loss',verbose=1, save_best_only=True)

'''
The training starts here.
'''
#model.fit_generator(myGene,steps_per_epoch=20000,epochs=5,callbacks=[model_checkpoint])









[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12653225740976784770
]


/Users/zifengli/OneDrive/Github/3D_MIP_ML/_Scripts/UNet/Training Module/model.py:62: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input = inputs, output = conv10)


'\nThe training starts here.\n'

In [9]:
test = next(myGene)

Found 30 images belonging to 1 classes.
Found 30 images belonging to 1 classes.


In [10]:
test[0][0][0][0]

array([0.02352941, 0.00392157, 0.        ], dtype=float32)